In [ ]:
import os
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import cv2 as cv

handrailWidth = 1600

folder_path = 'C:/Users/marco/Downloads/HKPC_Share/MTR_Handrail_Inspector2/notebook/'
file_list = os.listdir(folder_path)

arrays = []

csvCounter = 0

for file_name in file_list:
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        data = np.genfromtxt(file_path, delimiter=',')
        arrays.append(data)
        
        csvCounter+=1
        
print("No. of csv processed: ", csvCounter)

Z = np.concatenate(arrays, axis=0)

centerLine = Z.shape[1]/2
Z = Z[:,int(centerLine-handrailWidth/2):int(centerLine+handrailWidth/2)]

minPtX = Z.min(axis=0)
minPtY = Z.min(axis=1)
handrailSurfaceLvX = np.median(Z, axis=0)
handrailSurfaceLvY = np.median(Z, axis=1)

#Plot 2D Graph ###############################################################################################
fig = plt.figure(figsize=(24, 18))
plt.subplot(3, 1, 1)

plt.imshow(Z.T, interpolation='nearest', aspect='auto')
plt.axis('on')
plt.savefig("defect.jpg", bbox_inches='tight')

plt.subplot(3, 1, 2)
plt.plot(minPtX)

plt.subplot(3, 1, 3)
plt.plot(minPtY)

plt.show()

# Clustering - DBSCAN ###############################################################################################
labels = DBSCAN(eps=2, min_samples=300).fit_predict(Z)

print(np.where(labels!=0))
index = np.arange(0, Z.shape[0], 1)
print(index)

fig = plt.figure(figsize=(24, 6))
plt.plot(labels)

# Plot 3D Graph ###############################################################################################
X = np.arange(0, Z.shape[1], 1)
Y = np.arange(0, Z.shape[0], 1)
X, Y = np.meshgrid(X, Y)

Zinv = -Z

Xinv = np.arange(0, Zinv.shape[1], 1)
Yinv = np.arange(0, Zinv.shape[0], 1)
Xinv, Yinv = np.meshgrid(Xinv, Yinv)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(25, 25), subplot_kw={"projection": "3d"})
ax1.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax2.plot_surface(Xinv, Yinv, Zinv, cmap=cm.coolwarm, linewidth=0, antialiased=False)

plt.show()

In [ ]:
img = cv.imread('defect.jpg')

(imgh, imgw) = img.shape[:2]

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#cv.imshow("gray", gray)

ret,thresh = cv.threshold(gray,127,255,cv.THRESH_BINARY)
#cv.imshow("thresh", thresh)

contours,hierarchy = cv.findContours(thresh, cv.RETR_TREE,cv.CHAIN_APPROX_NONE)

count=0
font=cv.FONT_HERSHEY_COMPLEX

for cnt in contours:
    
    # find contour area
    contourArea = cv.contourArea(cnt)
    
    # find min rectangle
    rect = cv.minAreaRect(cnt) 
    (Rectx, Recty), (RectWidth, RectHeight), angle = rect
    box = cv.boxPoints(rect) 
    box = np.intp(box) 
    
    # find min circle
    (cx,cy),radius = cv.minEnclosingCircle(cnt)
    center = (int(cx),int(cy))
    radius = int(radius)  
                   
    # find font center for marking defect no.    
    fontCenter = (int(cx),int(cy+radius+30))
  
    if 10 < radius < 900:
                
        #cv.drawContours(img, cnt, -1, (0,255,255), 2)                     # draw contour
        #cv.drawContours(img, [box], 0, (0,0,255), 2)                      # draw min rectangle
        cv.circle(img,center,radius,(255,255,255),2)                      # draw min circle
        cv.putText(img,str(count+1),fontCenter,font,1,(255,255,255),2)    # mark defect no.
        
        print("Defect ", count+1, ":")
        print("Center Point: ", center)
        print("Contour Area(px): ",int(contourArea))
        print("Length(px): ",int(RectHeight))
        print("Width(px): ",int(RectWidth))
        print("Depth(px): N/A")        
        print("")
        count+=1

cv.putText(img,('Total defects: '+str(count)),(imgw//2, imgh//6),font,1,(255,255,255),2)

cv.imshow("result", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN


handrailWidth = 1600

folder_path = 'C:/Users/marco/Downloads/HKPC_Share/MTR_Handrail_Inspector2/notebook/'
file_list = os.listdir(folder_path)

csvCounter = 0
index = 0

for file_name in file_list:
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        Z = np.genfromtxt(file_path, delimiter=',')

        centerLine = Z.shape[1]/2
        Z = Z[:,int(centerLine-handrailWidth/2):int(centerLine+handrailWidth/2)]
        
        
        labels = DBSCAN(eps=2, min_samples=100).fit_predict(Z)
        
        index += Z.shape[0]
        csvCounter+=1
        
        defectLength = np.where(labels<0)
        print(defectLength)
    
   
        fig = plt.figure(figsize=(24, 6))
        plt.plot(labels)

        plt.show()
        

        csvCounter+=1
        
        
print("No. of csv processed: ", csvCounter)
print("Handrail Length: ", index)
